### 加载模型和数据集

In [1]:
import torch

print(torch.cuda.is_available())

from modelscope import AutoTokenizer
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq

tokenizer = AutoTokenizer.from_pretrained("./Qwen2.5-0.5B-Instruct", use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("./Qwen2.5-0.5B-Instruct", device_map="cuda", torch_dtype="auto")

True


d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
from datasets import Dataset
from swanlab.integration.huggingface import SwanLabCallback
from peft import LoraConfig, TaskType, get_peft_model
import swanlab

C:\Users\29277\AppData\Local\Temp\ipykernel_27496\2019269277.py:3: DeprecationWarning: The module 'huggingface' is deprecated and will be removed in future versions. Please update your imports to use 'transformers' instead.
  from swanlab.integration.huggingface import SwanLabCallback


In [3]:
train_df = pd.read_json('multi-class-1000x10.json')
train_ds = Dataset.from_pandas(train_df)

In [4]:
train_df

,文本,风险类别
0,您好，我是辉县消防队的采购负责人，需要订购一批鱼和牛蛙，请添加我的微信好友，并按指示转账34...,冒充军警购物类诈骗
1,尊敬的快手分期用户，您的账户因资料填写错误被临时冻结，请立即缴纳6000元认证金以解冻账户，...,虚假信用服务类
2,您被邀请加入了一个QQ群，只需为公众号点赞并发送截图即可领取红包。接下来，请下载Ruby A...,虚假网络投资理财类
3,您在知乎APP上看到的贷款信息可能不实，请谨慎对待网络贷款信息，建议通过正规渠道申请贷款。,无风险
4,您好，我们已获得您的视频聊天内容，如果不希望视频泄露给您的亲友，请立即转账至12345678...,网黑案件
...,...,...
9995,您好，我是银监会的工作人员，发现您的网上贷款存在违规记录，需要您立即处理以避免影响信用。请访...,虚假购物、服务类
9996,尊敬的用户，您好！通过斯达特软件加入的国风购物平台，现推出新品投资机会，选择品牌Ad+1、臻...,虚假网络投资理财类
9997,您好，我是京东客服，检测到您的京东APP借贷服务利率高于国家标准，请立即下载小鱼易连APP进...,冒充电商物流客服类
9998,您好，我是微信金融的客服，检测到您的微信贷款额度存在异常，需要立即注销，否则将影响您的个人征...,虚假信用服务类


In [4]:
category = "短信风险类别选项有：无风险|冒充电商物流客服类|虚假网络投资理财类|虚假信用服务类|虚假购物、服务类|冒充公检法及政府机关类|冒充领导、熟人类|网络婚恋、交友类|冒充军警购物类诈骗|网黑案件"
prompt = '''在这个任务中，你是一位资深的反诈骗网络安全分析师，你的职责是利用你的专业知识和对网络诈骗行为的深刻理解，从短信文本中识别出可能存在的欺诈行为和风险类别。你的工作对于提前预警潜在的网络诈骗，保护用户财产安全和个人信息不被侵犯具有重要意义。现在，请仔细审查以下短信文本，并运用你的专业判断，给出短信的风险类别判断结果。(%s)'''%category

def process_func(example):
    """
    将数据集进行预处理
    """
    MAX_LENGTH = 384 
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(
        f"<|im_start|>system\n{prompt}<|im_end|>\
        \n<|im_start|>user\n{example['文本']}<|im_end|>\n<|im_start|>assistant\n",
        add_special_tokens=False,
    )
    response = tokenizer(f"{example['风险类别']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = (
        instruction["attention_mask"] + response["attention_mask"] + [1]
    )
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}  


In [8]:
train_dataset = train_ds.map(process_func, remove_columns=train_ds.column_names)
train_dataset.save_to_disk("multi-class-1000x10")

d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\datasets\utils\_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]
Saving the dataset (1/1 shards): 100%|██████████| 10000/10000 [00:00<00:00, 162665.76 examples/s]


In [5]:
from datasets import load_from_disk
train_dataset = load_from_disk("multi-class-1000x10")

In [6]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})

### 模型微调

In [7]:
model = AutoModelForCausalLM.from_pretrained("./Qwen2.5-0.5B-Instruct", device_map="auto", torch_dtype="auto")
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "lm_head"],
    inference_mode=False,  # 训练模式
    r=8,  # Lora 秩
    lora_alpha=32,  # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1,  # Dropout 比例
)

# for param in model.parameters():
#     param.requires_grad = True
    
model = get_peft_model(model, config)

In [8]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=896, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=896, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
   

In [9]:
model.print_trainable_parameters()

trainable params: 5,621,760 || all params: 499,654,528 || trainable%: 1.1251


In [10]:
args = TrainingArguments(
    output_dir="./output/multi-class-20epoch",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=2500,
    num_train_epochs=20,
    save_steps=2500,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True,
    report_to="none",
)

swanlab_callback = SwanLabCallback(
    project="Qwen2.5-0.5B-Instruct-Finetuning",
    experiment_name="Qwen2.5-0.5B-Instruct-multi-class",
    description="Qwen2.5-0.5B-Instruct模型微调。",
    config={
        "model": "qwen/Qwen2.5-0.5B-Instruct",
        "dataset": "FGRC-SCD电信诈骗数据集",
    }
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    callbacks=[swanlab_callback],
)

In [11]:
trainer.train()

swanlab: swanlab version 0.4.2 is available!  Upgrade: `pip install -U swanlab`
swanlab: \ Getting project...                                                      

d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\swanlab\api\http.py:114: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  if (self.sid_expired_at - datetime.utcnow()).total_seconds() <= self.REFRESH_TIME:


swanlab: Tracking run with swanlab version 0.3.21                                  
swanlab: Run data will be saved locally in d:\fudan\LLM\FineTune-Qwen\swanlog\run-20250116_223835-a3b1799d
swanlab: 👋 Hi tangerine, welcome to swanlab!
swanlab: Syncing run Qwen2.5-0.5B-Instruct-multi-class0116 to the cloud
swanlab: 🌟 Run `swanlab watch d:\fudan\LLM\FineTune-Qwen\swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@tangerine/Qwen2.5-0.5B-Instruct-Finetuning
swanlab: 🚀 View run at https://swanlab.cn/@tangerine/Qwen2.5-0.5B-Instruct-Finetuning/runs/4k860rnrg1gvdy1b7vsmn


  0%|          | 0/12500 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\torch\utils\checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:580: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions

{'loss': 0.3195, 'grad_norm': 5.1099629402160645, 'learning_rate': 8e-05, 'epoch': 4.0}


...

{'loss': 0.1294, 'grad_norm': 5.7159833908081055, 'learning_rate': 0.0, 'epoch': 20.0}


100%|██████████| 12500/12500 [3:54:09<00:00,  1.12s/it]

swanlab: Step 12500 on key train/epoch already exists, ignored.
{'train_runtime': 14055.3094, 'train_samples_per_second': 14.229, 'train_steps_per_second': 0.889, 'train_loss': 0.17544189697265625, 'epoch': 20.0}


TrainOutput(global_step=12500, training_loss=0.17544189697265625, metrics={'train_runtime': 14055.3094, 'train_samples_per_second': 14.229, 'train_steps_per_second': 0.889, 'total_flos': 1.0339122852646502e+17, 'train_loss': 0.17544189697265625, 'epoch': 20.0})

d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\swanlab\api\http.py:114: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  if (self.sid_expired_at - datetime.utcnow()).total_seconds() <= self.REFRESH_TIME:


In [12]:
swanlab.finish()

swanlab: 🌟 Run `swanlab watch d:\fudan\LLM\FineTune-Qwen\swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@tangerine/Qwen2.5-0.5B-Instruct-Finetuning
swanlab: 🚀 View run at https://swanlab.cn/@tangerine/Qwen2.5-0.5B-Instruct-Finetuning/runs/4k860rnrg1gvdy1b7vsmn
swanlab: \ Updating experiment status...                                           

d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\executing\executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
d:\Programs\anaconda3\envs\cuda121-2\Lib\ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)


ApiError: (409, 'Conflict')